In [116]:
import sys
sys.path.append('..')
from configure.settings import DBSelector
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
from pyecharts.charts import Scatter
import pyecharts.options as opts
from pyecharts.charts import Line, Grid
from pyecharts.commons.utils import JsCode
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import tree,preprocessing
from sklearn.feature_extraction import DictVectorizer
%matplotlib inline

background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
)
area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
)

In [107]:
def get_bondPremRatio(conv_price,current_price):
    inner_value = get_conv_value(conv_price,current_price)
    return (100/inner_value-1)*100

def convert_permratio_inner(bondPremRatio):
    return 100/(bondPremRatio/100+1)

# 转股价值
def get_conv_value(conv_price,current_price):
    return 100/conv_price*current_price

In [155]:
get_bondPremRatio(4.05,3.95)

2.5316455696202667

In [108]:
# load data
file = '/home/xda/hub/convertible_bond/data/new_bond_info.xlsx'
data = pd.read_excel(file,index_col=0)

In [109]:
data.drop('neg_market_value',axis=1,inplace=True)

In [110]:
data.head()

,bondPremRatio,call_trigger_date,call_trigger_rate,closePriceBond,code,coupon,firstDate,industry,put_trigger_date,put_trigger_rate,rate,remainSize,secShortNameBond
0,2.1033,30,130,104.180,110041,0.4,20180109,公用事业,30.0,70.0,AAA,18.7522,蒙电转债
1,12.0147,30,130,112.379,128024,0.2,20180112,银行,-1.0,-1.0,AAA,100.0000,宁行转债
2,7.9547,30,130,101.760,110042,0.2,20180115,国防军工,30.0,70.0,AAA,24.0000,航电转债
3,-0.5855,30,130,98.080,113017,0.3,20180115,传媒,30.0,70.0,AA+,15.6000,吉视转债
4,2.8069,30,130,124.530,128029,0.3,20180116,轻工制造,30.0,70.0,AA+,12.0000,太阳转债


In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458 entries, 0 to 457
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bondPremRatio      458 non-null    float64
 1   call_trigger_date  458 non-null    int64  
 2   call_trigger_rate  458 non-null    int64  
 3   closePriceBond     458 non-null    float64
 4   code               458 non-null    int64  
 5   coupon             458 non-null    float64
 6   firstDate          458 non-null    int64  
 7   industry           458 non-null    object 
 8   put_trigger_date   454 non-null    float64
 9   put_trigger_rate   454 non-null    float64
 10  rate               458 non-null    object 
 11  remainSize         458 non-null    float64
 12  secShortNameBond   458 non-null    object 
dtypes: float64(6), int64(4), object(3)
memory usage: 50.1+ KB


In [112]:
rate_map = {
    'A':0,
    'A+':1,
    'AA-':2,
    'AA':3,
    'AA+':4,
    'AAA':5
}

data['rate'] = data['rate'].map(lambda x:rate_map.get(x))

In [113]:
industry_map = {
'化工':0,
'医药生物':    1,
'机械设备':    2,
'电子':      3,
'电气设备':   4,
'轻工制造':    5,
'汽车':      6,
'有色金属':   7,
'计算机':     8,
'建筑装饰':    9,
'公用事业':    10,
'农林牧渔':    11,
'银行':    12,
'纺织服装':    13,
'食品饮料':    14,
'国防军工':     15,
'通信':       16,
'非银金融':     17,
'交通运输':     18,
'家用电器':     19,
'建筑材料':     20,
'采掘':       21,
'钢铁':       22,
'传媒':       23,
'商业贸易':    24,
'休闲服务':  25,
}
data['industry']=data['industry'].map(lambda x:industry_map.get(x))

In [114]:
data['innerValue'] = data['bondPremRatio'].map(lambda x:convert_permratio_inner(x))

In [115]:
train_data = data.copy()
train_data.drop('secShortNameBond',inplace=True,axis=1)
train_data.drop('bondPremRatio',inplace=True,axis=1)

data_source = train_data.copy()
target = data_source['closePriceBond']
data_source.drop('closePriceBond',axis=1,inplace=True)
data_source.drop('firstDate',axis=1,inplace=True)

In [117]:
data_source.isnull().any()

call_trigger_date    False
call_trigger_rate    False
code                 False
coupon               False
industry             False
put_trigger_date      True
put_trigger_rate      True
rate                 False
remainSize           False
innerValue           False
dtype: bool

In [118]:
data_source['put_trigger_date'].fillna(0,inplace=True)

In [119]:
data_source['put_trigger_rate'].fillna(0,inplace=True)

In [120]:
data_source.drop('code',axis=1,inplace=True)

In [123]:
data_source[data_source['industry'].isnull()]

,call_trigger_date,call_trigger_rate,coupon,industry,put_trigger_date,put_trigger_rate,rate,remainSize,innerValue


In [124]:
X_train,X_test,y_train,y_test = train_test_split(data_source,target,random_state=4)

In [125]:
rfr = RandomForestRegressor()

In [126]:
from pprint import pprint

In [127]:
pprint(rfr.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [86]:
rf_param_grid ={
    # 'criterion':['entropy','gini'],
    'max_depth':[5,6,7,8],
    'n_estimators':[11,13,15],
    # 'max_features':[0.3,0.4,0.5],
    'min_samples_split':[4,8,12,16]
}

In [156]:
from sklearn.metrics import r2_score

In [99]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 8.51294807043729
Mean Squared Error: 120.97083543581468
Root Mean Squared Error: 10.99867425810105


In [100]:
import sklearn.metrics as metrics
metrics.classification_report(y_test,y_pred)

ValueError: continuous is not supported

In [128]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 150, num = 10)]
# Number of features to consider at every split
max_features = [3,4,5,6,6]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(3, 10, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3,4,5,6]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [129]:
rf = RandomForestRegressor()
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rfsv = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 200, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [130]:
rfsv.fit(X_train,y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=200,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [3, 10, None],
                                        'max_features': [3, 4, 5, 6, 6],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 3, 4, 5, 6],
                                        'n_estimators': [10, 25, 41, 56, 72, 87,
                                                         103, 118, 134, 150]},
                   random_state=42, verbose=2)

In [132]:
rfsv.best_params_

{'n_estimators': 150,
 'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_features': 3,
 'max_depth': None,
 'bootstrap': False}

In [133]:
rf4=RandomForestRegressor(**rfsv.best_params_)

In [134]:
rf4.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, max_features=3, min_samples_leaf=4,
                      min_samples_split=4, n_estimators=150)

In [135]:
y_pred4 = rf4.predict(X_test)

In [136]:
evaluate_(y_test,y_pred4)

mean absolute error= 9.78
mean squared error= 144.76
median absolute error= 8.14
explained variance score= 0.04
R2 score= 0.04


In [ ]:
rf4

In [125]:
y_pred = rfsv.predict(X_test)

In [137]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [138]:
evaluate(rfsv,X_test,y_test)

Model Performance
Average Error: 9.8196 degrees.
Accuracy = 91.30%.


91.29786968451995

In [139]:
def evaluate_(y_test,y_test_pre):
    import sklearn.metrics as sm
    print('mean absolute error=',round(sm.mean_absolute_error(y_test,y_test_pre),2))
    print('mean squared error=',round(sm.mean_squared_error(y_test,y_test_pre),2))
    print('median absolute error=',round(sm.median_absolute_error(y_test,y_test_pre),2))
    print('explained variance score=',round(sm.explained_variance_score(y_test,y_test_pre),2))
    print('R2 score=',round(sm.r2_score(y_test,y_test_pre),2))

In [128]:
evaluate_(y_test,y_pred)

mean absolute error= 9.37
mean squared error= 142.86
median absolute error= 8.22
explained variance score= 0.05
R2 score= 0.05


In [130]:
rfsv.best_params_

{'n_estimators': 244,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': True}

In [ ]:
rfsv.feature_importances_

In [140]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10],
    'max_features': [4,5,6],
    'min_samples_leaf': [2,3, 4, 5,6],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [30]
}
# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [38]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 4,
 'min_samples_leaf': 6,
 'min_samples_split': 8,
 'n_estimators': 30}

In [141]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [5, 10],
                         'max_features': [4, 5, 6],
                         'min_samples_leaf': [2, 3, 4, 5, 6],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [30]},
             verbose=2)

In [142]:
y_pred_grid = grid_search.predict(X_test)

In [143]:
rf_ = RandomForestRegressor(**grid_search.best_params_)
rf_.fit(X_train,y_train)
y_pred_grid = rf_.predict(X_test)
evaluate_(y_test,y_pred_grid)

mean absolute error= 10.0
mean squared error= 148.04
median absolute error= 8.76
explained variance score= 0.02
R2 score= 0.02


In [34]:
evaluate_(y_test,y_pred_grid)

mean absolute error= 6.62
mean squared error= 73.79
median absolute error= 5.1
explained variance score= 0.02
R2 score= 0.01


In [152]:

importances = rf_.feature_importances_
importances=list(map(lambda x:round(x*100,0),importances))
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %.2f" % (f + 1, 30, X_train.columns[indices[f]], importances[indices[f]]))


 1) innerValue                     38.00
 2) industry                       22.00
 3) remainSize                     21.00
 4) coupon                         12.00
 5) rate                           6.00
 6) put_trigger_rate               0.00
 7) put_trigger_date               0.00
 8) call_trigger_rate              0.00
 9) call_trigger_date              0.00


In [147]:
X_train.shape[1]

9

In [65]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 50}

In [81]:
rf_ = RandomForestRegressor(
    n_estimators=50,
    max_depth=110,
    max_features=3,
    bootstrap=True,
    min_samples_split=8,
    min_samples_leaf=3
)

In [133]:
rf2=RandomForestRegressor(**)

In [134]:
rf2.fit(X_train,y_train)

RandomForestRegressor(min_samples_leaf=2, min_samples_split=10,
                      n_estimators=244)

In [135]:
y_pred_3 = rf2.predict(X_test)

In [82]:
rf_.fit(X_train,y_train)

RandomForestRegressor(max_depth=110, max_features=3, min_samples_leaf=3,
                      min_samples_split=8, n_estimators=50)

In [83]:
y_pred_2 = rf_.predict(X_test)

In [136]:
evaluate_(y_test,y_pred_3)

mean absolute error= 9.45
mean squared error= 144.96
median absolute error= 8.34
explained variance score= 0.04
R2 score= 0.04


In [85]:
rf_.feature_importances_

array([0.23106684, 0.        , 0.00549895, 0.06502069, 0.13070471,
       0.33268429, 0.00359227, 0.00752537, 0.05340304, 0.17050384])

In [47]:
feature_list_ = list(feature_list[0].keys())

In [52]:
len(importances)

38

In [53]:
X_train.shape

(343, 38)

In [50]:
importances = rf_.feature_importances_
indices = np.argsort(importances)[::-1] # 大到小
for i in range(len(feature_list_)):
    # print(i)
    # print("%2d) %-*s %f" % (i + 1, 30, feature_list_[indices[i]], importances[indices[i]]))
    print(indices[i])

37
2
16
18
35
34
33
15


In [86]:
len(importances)

11

In [138]:
X_train

,call_trigger_date,call_trigger_rate,coupon,industry,neg_market_value,put_trigger_date,put_trigger_rate,rate,remainSize,innerValue
191,30,130,0.4,3,1.001429e+10,30.0,70.0,3,26.600177,87.797546
45,30,130,0.5,16,2.880305e+09,30.0,70.0,3,10.000000,102.371015
241,30,130,0.5,0,2.087673e+09,30.0,70.0,2,3.990000,85.508308
377,30,130,0.4,6,3.355806e+09,30.0,70.0,2,8.864800,90.721792
325,30,130,0.3,7,4.013141e+10,30.0,70.0,3,21.080000,70.273723
...,...,...,...,...,...,...,...,...,...,...
385,30,130,0.4,3,1.395576e+09,30.0,60.0,2,6.730000,82.376598
197,30,130,0.4,13,5.733447e+09,30.0,70.0,3,6.500000,93.553421
439,30,130,0.2,20,4.343613e+10,30.0,70.0,4,15.000000,91.574169
174,30,130,0.3,4,3.444622e+09,30.0,70.0,3,8.800000,90.144014


In [97]:
get_bondPremRatio(9.02,13.90)

-35.10791366906476

In [156]:

input_data = {
'innerValue':convert_permratio_inner(2.5316455696202667),
'call_trigger_date':30,
'call_trigger_rate':130, # 强赎
'coupon':0.2,
'industry':industry_map.get('公用事业'),
# 'neg_market_value':21042642880,
'put_trigger_date':30,
'put_trigger_rate':70, # triggerRate 回售触发票面比例（%）
'rate':rate_map.get('AA+'),
'remainSize':30.00*10**9,
}

In [157]:
df_real = pd.DataFrame([input_data])

In [158]:
df_real

,innerValue,call_trigger_date,call_trigger_rate,coupon,industry,put_trigger_date,put_trigger_rate,rate,remainSize
0,97.530864,30,130,0.2,10,30,70,4,3.000000e+10


In [160]:
rf_.predict(df_real)[0]

118.97838485149109

In [205]:
rf4.feature_importances_

array([0.        , 0.00569037, 0.11630724, 0.20587691, 0.00887675,
       0.00978199, 0.07112373, 0.20523529, 0.37710772])

In [104]:
100/(-35.107914/100+1)

154.1019963512962

In [106]:
convert_permratio_inner(-35.107914)

154.1019963512962

In [65]:
data_source.head()

,call_trigger_date,call_trigger_rate,coupon,industry,put_trigger_date,put_trigger_rate,rate,remainSize,innerValue
0,30,130,0.4,公用事业,30.0,70.0,AAA,18.7522,97.940027
1,30,130,0.2,银行,-1.0,-1.0,AAA,100.0000,89.273997
2,30,130,0.2,国防军工,30.0,70.0,AAA,24.0000,92.631446
3,30,130,0.3,传媒,30.0,70.0,AA+,15.6000,100.588948
4,30,130,0.3,轻工制造,30.0,70.0,AA+,12.0000,97.269736


In [66]:
target.head()

0    104.180
1    112.379
2    101.760
3     98.080
4    124.530
Name: closePriceBond, dtype: float64

In [67]:
data_source.columns

Index(['call_trigger_date', 'call_trigger_rate', 'coupon', 'industry',
       'put_trigger_date', 'put_trigger_rate', 'rate', 'remainSize',
       'innerValue'],
      dtype='object')

In [68]:
feature_list=[]
label_list =[]
feature_name = data_source.columns
label_name = target
for index,row in data_source.iterrows():
    feature={}
    for i in range(len(feature_name)):
        feature[feature_name[i]]=row[feature_name[i]]
    feature_list.append(feature)
    # label_list.append(row[label_name])

In [71]:
feature_list[0]

{'call_trigger_date': 30,
 'call_trigger_rate': 130,
 'coupon': 0.4,
 'industry': '公用事业',
 'put_trigger_date': 30.0,
 'put_trigger_rate': 70.0,
 'rate': 'AAA',
 'remainSize': 18.7522,
 'innerValue': 97.94002740361967}

In [69]:
len(feature_list)

458

In [70]:
len(data_source)

458

In [47]:
feature_list[0]

{'call_trigger_date': 30,
 'call_trigger_rate': 130,
 'coupon': 0.4,
 'industry': '公用事业',
 'put_trigger_date': 30.0,
 'put_trigger_rate': 70.0,
 'rate': 'AAA'}

In [72]:
vec = DictVectorizer()
X = vec.fit_transform(feature_list).toarray()

In [42]:
feature_list[0]

{'call_trigger_date': 30,
 'call_trigger_rate': 130,
 'coupon': 0.4,
 'industry': '公用事业',
 'put_trigger_date': 30.0,
 'put_trigger_rate': 70.0,
 'rate': 'AAA',
 'remainSize': 18.7522}

In [101]:
vec = DictVectorizer(sparse=False)
X = vec.fit_transform(data_source.to_dict(orient='records'))

In [84]:
data_source.columns

Index(['call_trigger_date', 'call_trigger_rate', 'coupon', 'industry',
       'put_trigger_date', 'put_trigger_rate', 'rate', 'remainSize',
       'innerValue'],
      dtype='object')

In [105]:
vec.feature_names_

['call_trigger_date',
 'call_trigger_rate',
 'coupon',
 'industry',
 'innerValue',
 'put_trigger_date',
 'put_trigger_rate',
 'rate',
 'remainSize']

In [83]:
len(X[0])

39

In [104]:
len(X[0])

9

In [27]:
label = preprocessing.LabelBinarizer()
Y = label.fit_transform(label_list)

ValueError: Unknown label type: ([97.94002740361967, 89.27399707359838, 92.63144633813998, 100.58894829225113, 97.2697357862167, 76.9847831877551, 80.7957410948954, 88.98800174772435, 87.2176437804662, 83.98731455600947, 92.42810479868234, 93.78452442805508, 92.62175129207343, 84.7340579225073, 98.91255536630287, 95.34888155761932, 84.75890752548914, 91.50051697792094, 90.18287282952372, 99.10734018693627, 90.97591685529007, 91.94725905220766, 94.37781894747968, 95.2580540684715, 99.83218210188673, 88.49886455956769, 95.9515176171784, 96.4258783915392, 94.91502258028387, 90.53206600511868, 99.28228833760744, 91.08596328871336, 92.0074084365273, 90.23332533264515, 84.24670805988256, 83.90430210918635, 91.58557527189468, 92.75233272116793, 94.22779380697247, 92.88595740807308, 100.56568196103079, 89.3814801573114, 100.70949841634314, 98.40204912427097, 99.73560092195589, 102.37101508027423, 97.30844831948309, 97.09143197240273, 94.89989010592726, 94.79041365588616, 97.87417296323846, 95.

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,label_list,random_state=4)

In [90]:
#-*- coding:utf-8 -*-

#定义一组字典列表，用来表示多个数据样本（每个字典代表一个数据样本）
measurements = [{'city':1,'country':'CN','temperature':33.},{'city':2,'country':'UK','temperature':12.},{'city':3,'country':'USA','temperature':18.}]
#从sklearn.feature_extraction导入DictVectorizer
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
#输出转化后的特征矩阵
print(vec.fit_transform(measurements).toarray())
#输出各个维度的特征含义
print(vec.get_feature_names())


[[ 1.  1.  0.  0. 33.]
 [ 2.  0.  1.  0. 12.]
 [ 3.  0.  0.  1. 18.]]
['city', 'country=CN', 'country=UK', 'country=USA', 'temperature']


In [154]:
with open('new_bond_tree.dot','w') as f:
    tree.export_graphviz(rf_.estimators_[0],feature_names=list(data_source.columns),out_file=f)

In [163]:
import joblib
joblib.dump(rf_, "train_model.m")
# clf = joblib.load("train_model.m") ＃调用


['train_model.m']